In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import osmnx as ox
import numpy as np
from network_construction.network import source_treatment

In [2]:
G = ox.load_graphml("C:\\Users\mbans\Desktop\CMOR492-DWS\DWS\\network_construction\\road_net_2.graphml")
source_nodes, treatment_nodes = source_treatment(G, 20)  # <-- Specify # starting points for treatment node algorithm

In [46]:
### MODEL PARAMETERS

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe

SR = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = 1.8
    SR[node] = 1.8

total_flow = sum(SR.values())

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 1000
    CAP[node] = 1000

Vmin = 0.6
Vmax = 3

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 10000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width

In [113]:
m = gp.Model()

### DECISION VARIABLES

x = m.addVars(Path.keys(), vtype=GRB.BINARY, name=[f'x: {i} --> {j}' for i, j in Path.keys()])  # Path ij used
y = m.addVars(treatment_nodes, vtype=GRB.BINARY, name=[f'y_{j}' for j in treatment_nodes])  # treatment at node j
z = m.addVars(G.edges, vtype=GRB.BINARY, name=[f'z: {e}' for e in G.edges])  # edge e used

d_es_names = []
for e in G.edges:
    for s in D:
        d_es_names.append(f"d_{e}_{s}")

d_es = m.addVars(G.edges, D, vtype=GRB.BINARY, name=d_es_names)  # Pipe size s at edge e

# Recourse Amount
r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'r_{i}' for i in source_nodes])  # flow handled at trucking at edge e

# Flow in Edge e
Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name=[f'q_{e}' for e in G.edges])  # Flow in Edge e

# Node Elevation
el = m.addVars(G.nodes, vtype=GRB.CONTINUOUS, name=[f'el_{v}' for v in G.nodes])  # Elevation at node el_v

# Path Flow
p = m.addVars(list(Path.keys()), vtype=GRB.CONTINUOUS, lb = 0.0, name=[f"p: {i} -- > {j}" for i, j in Path.keys()])

m.update()

In [114]:
### CONSTRAINTS

# NODE PRODUCTION MINUS RECOURSE
node_prod_rec = m.addConstrs(p[i, j] >= (SR[i] * x[i, j]) - r[i] for i, j in Path.keys())

# TREATMENT CAPACITY
# treat_cap = m.addConstrs(gp.quicksum(SR[i] * x[i, j] for i in source_nodes) <= CAP[j]*y[j] for j in treatment_nodes)
treat_cap = m.addConstrs(gp.quicksum(p[i, j] for i in source_nodes) <= CAP[j] * y[j] for j in treatment_nodes)

#  NODE ASSIGNMENT
node_assign = m.addConstrs(gp.quicksum(x[i, j] for j in treatment_nodes) == 1 for i in source_nodes)

# PIPE SIZING
pipe_sizing = m.addConstrs(gp.quicksum(d_es[*e, s] for s in D) == z[e] for e in G.edges)


# FLOW DEFINITION
def is_sublist(short_list, long_list):
    for i in range(len(long_list) - len(short_list) + 1):
        if long_list[i:i + len(short_list)] == short_list:
            return True
    return False

flow_def = m.addConstrs(Q[e] == gp.quicksum(p[i, j] for i, j in Path.keys() if is_sublist(list((e[0], e[1])),Path[i,j])) for e in G.edges)

# EDGE ACTIVATION
Path_e = {}  # Use this for Edge Activiation Constraint
for e, p in Path.items():
    Path_e[e] = [(p[l - 1], p[l]) for l in range(1, len(p))]

# For stuff indexed by edges, we have to include the extra zero that OSMNX added---at some point I'll try to get rid of it lol

edge_activate = m.addConstrs(gp.quicksum(z[*e, 0] for e in Path_e[i, j]) >= NLinks[i, j] * x[i, j] for i, j in Path)

# MIN/MAX SLOPE
min_slope = m.addConstrs(el[e[0]] - el[e[1]] >= (LE[e[0], e[1], 0] * Smin) - (M * (1 - z[e])) for e in G.edges)
max_slope = m.addConstrs(el[e[0]] - el[e[1]] <= (LE[e[0], e[1], 0] * Smax) + (M * (1 - z[e])) for e in G.edges)

# FLOW VELOCITY LIMIT
flow_vel = m.addConstrs(Q[e] <= Vmax * gp.quicksum((np.pi / 8) * (s**2) * (d_es[*e, s]) for s in D) for e in G.edges)

# PIPES UNDERGROUND
underground = m.addConstrs(el[i] <= EL[i] for i in source_nodes)

# MANNING'S EQUATION

# manning = m.addConstrs(
#     (LE[u, v, 0] * gp.quicksum(d_es[u, v, 0, s] * ((Q[u, v, 0]**2) / ((11.9879 * (s**(8/3)))**2)) for s in D)
#      <= el[u] - el[v] for u, v, _ in G.edges),
#     name="manning")

m.update()

In [54]:
# OLD MANNING

# Create new variables for the squared terms
# Q_squared = m.addVars(G.edges, name="Q_squared")
#
# manning = m.addConstrs(
#     (LE[u, v] * gp.quicksum(d_es[u, v, s] * Q_squared[u, v] / (11.9879 * (s**(8/3)))**2 for s in D)
#      <= el[u] - el[v] for u, v in G.edges),
#     name="manning")

# Add constraints to enforce Q_squared[u, v] = Q[u, v]^2
# q_square = m.addConstrs((Q_squared[u, v] == Q[u, v] * Q[u, v] for u, v in G.edges), name="Q_squared_def")


In [115]:
### OBJECTIVE EPXR 1: TREATMENT COSTS

treat_cost = gp.LinExpr()
for j in treatment_nodes:
    treat_cost.addTerms(TR, y[j])
    for i in source_nodes:
        treat_cost.addTerms(TRFlow * SR[i], x[i, j])

# OBJECTIVE EXPR 2: EXCAVATION COSTS
excav_cost_f = lambda u, v: gp.QuadExpr(CE * (((EL[u] - el[u]) + (EL[v] - el[v])) / 2) * LE[u, v, 0] * gp.quicksum(s + ((2*W)*d_es[u, v, 0, s]) for s in D))

# OBJECTIVE EXPR 3: BEDDING COSTS
bed_cost_f = lambda u, v: gp.LinExpr(CB *  LE[u, v, 0] * gp.quicksum(s + ((2*W)*d_es[u, v, 0, s]) for s in D))
# OBJECTIVE EXPR 4: PIPE COSTS
pipe_cost_f = lambda u, v: gp.LinExpr(LE[u, v, 0] * gp.quicksum(CP[s] * d_es[u, v, 0, s] for s in D))

excav_bed_cost = gp.quicksum(excav_cost_f(u, v) + bed_cost_f(u, v) + pipe_cost_f(u, v) for u, v, _ in G.edges)

# OBJECTIVE EXPR 5: RECOURSE TRUCKING

rec_cost = gp.LinExpr()
for i in source_nodes:
    rec_cost.addTerms(CT, r[i])
m.setObjective(treat_cost + excav_bed_cost + rec_cost, GRB.MINIMIZE)

m.update()

In [83]:
print(f"Model has {m.NumVars} variables and {m.NumConstrs} constraints.")

Model has 9966 variables and 8433 constraints.


In [116]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8433 rows, 9966 columns and 93615 nonzeros
Model fingerprint: 0x7ea2cb83
Model has 6054 quadratic objective terms
Variable types: 3649 continuous, 6317 integer (6317 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+06]
  Objective range  [2e+02, 1e+07]
  QObjective range [3e+02, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 941 rows and 995 columns
Presolve time: 0.23s
Presolved: 12640 rows, 14119 columns, 94407 nonzeros
Variable types: 8408 continuous, 5711 integer (5711 binary)
Found heuristic solution: objective 3.491441e+09
Found heuristic solution: objective 3.427743e+09
Found heuristic solution: objective -1.72521e+11
Deterministic concurrent LP optimizer: pr

In [ ]:
for v in m.getVars():
    if v.X > 0:
        print(v.varName, v.X)

In [ ]:
for v in Q:
    var = Q[v]
    if var.X > 0:
        print(var.varName, var.X)